In [52]:
# coding: utf-8
import requests
import time
import math
import calendar
import json

#ur.urlopen('https://www.zhihu.com')

#requests.get('https://delta.sapanywhere.io/api/datasources/proxy/2/query?db=k8smetrics&q=SHOW%20TAG%20VALUES%20FROM%20%22uptime%22%20WITH%20KEY%20%3D%20%22namespace_name%22&epoch=ms')
#ur.urlopen('https://delta.sapanywhere.io/api/datasources/proxy/2/query?db=k8smetrics&q=SHOW%20TAG%20VALUES%20FROM%20%22uptime%22%20WITH%20KEY%20%3D%20%22namespace_name%22&epoch=ms')

def constructUri(feature,aggregate,startTime,endTime):
    uriPre = 'https://delta.sapanywhere.io/api/datasources/proxy/2/query?db=k8smetrics&q=SELECT%20AGGREGATE(%22value%22)%20FROM%20%22FEATURE%22%20WHERE%20%22type%22%20%3D%20%27pod_container%27%20AND%20%22namespace_name%22%20%3D~%20%2Fus%24%2F%20AND%20%22pod_name%22%20%3D~%20%2F%5Eattachment%2F%20AND%20%22container_name%22%20%3D~%20%2F%5Eattachment%24%2F%20AND%20time%20%3E%3D%20'
    uriPre = uriPre.replace('FEATURE',feature)
    uriPre = uriPre.replace('AGGREGATE',aggregate)
    startTime = str(startTime)
    uriMid = 's%20and%20time%20%3C%20'
    endTime = str(endTime)
    uriEnd = 's%20GROUP%20BY%20time(60s)%2C%20%22container_name%22%2C%20%22namespace_name%22%2C%20%22pod_name%22%20fill(null)' 
    uri = uriPre+startTime+uriMid+endTime+uriEnd
    return uri

def constructNetworkUri(feature,aggregate,startTime,endTime):
    uriPre = 'https://delta.sapanywhere.io/api/datasources/proxy/2/query?db=k8smetrics&q=SELECT%20AGGREGATE(%22value%22)%20FROM%20%22FEATURE%22%20WHERE%20%22type%22%20%3D%20%27pod%27%20AND%20%22namespace_name%22%20%3D~%20%2Fus%24%2F%20AND%20%22pod_name%22%20%3D~%20%2F%5Eattachment%2F%20AND%20time%20%3E%3D%20'
    uriPre = uriPre.replace('FEATURE',feature)
    uriPre = uriPre.replace('AGGREGATE',aggregate)
    startTime = str(startTime)
    uriMid = 's%20and%20time%20%3C%20'
    endTime = str(endTime)
    uriEnd = 's%20GROUP%20BY%20time(60s)%2C%20%22container_name%22%2C%20%22namespace_name%22%2C%20%22pod_name%22%20fill(null)' 
    uri = uriPre+startTime+uriMid+endTime+uriEnd
    return uri


def requestData(uri):
    r = requests.get(uri, cert=('./key/idKey.crt','./key/idKey.key'))
    return r.text

def getCurrentJsonData():
    interval =60
    return getJsonData(interval)

def getJsonData(intervalInSec):
    
    endTime = calendar.timegm(time.gmtime())
    endTime = math.floor(endTime/60)*60-1  # in minute

    startTime = endTime-intervalInSec+1

    
    cpuData = requestData(constructUri('cpu%2Fusage_rate','sum',startTime,endTime))
    memoryData = requestData(constructUri('memory%2Fusage','max',startTime,endTime))
    fileSystemData = requestData(constructUri('filesystem%2Fusage','sum',startTime,endTime))
    networkData = requestData(constructNetworkUri('network%2Ftx_rate','sum',startTime,endTime))

    cpuJson = json.loads(cpuData)
    memoryJson = json.loads(memoryData)
    fileSystemJson = json.loads(fileSystemData)
    networkJson = json.loads(networkData)

    
    print (cpuJson)
    print (memoryJson)
    print (fileSystemJson)
    print (networkJson)
   
    
    metricsData = dict()
    pods = list()
    metricsData['pods'] = pods
    
    series = list()
    result = cpuJson['results'][0]
    
    if 'series' in result :
        series = result['series'] 
        
    for serie in series:
        pod = dict()
    
        #init podName when processing the first feature(cup)
        pod['podName'] = serie['tags']['pod_name']
        data = list()
        for value in serie['values'] :
            cpuData = dict()
            cpuData['time'] = value[0]
            cpuData['cpu'] = value[1]
            data.append(cpuData)
    
        pod['data']=data   
        pods.append(pod)    
        
    series = list()
    result = memoryJson['results'][0]

    if 'series' in result :
        series = result['series'] 
    
    for serie in series:    
        podName = serie['tags']['pod_name']
        for pod in metricsData['pods']:
            if pod['podName'] == podName:
                d = pod['data']
                dataIndex = 0
                for value in serie['values']:
                    d[dataIndex]['memory'] = value[1]
                    dataIndex+=1
    
    series = list()
    result = fileSystemJson['results'][0]

    if 'series' in result :
        series = result['series'] 
    
    for serie in series:    
        podName = serie['tags']['pod_name']
        for pod in metricsData['pods']:
            if pod['podName'] == podName:
                d = pod['data']
                dataIndex = 0
                for value in serie['values']:
                    d[dataIndex]['fileSystem'] = value[1]
                    dataIndex+=1
    
    series = list()
    result = networkJson['results'][0]

    if 'series' in result :
        series = result['series']

    for serie in series:    
        podName = serie['tags']['pod_name']
        for pod in metricsData['pods']:
            if pod['podName'] == podName:
                d = pod['data']
                dataIndex = 0
                for value in serie['values']:
                    d[dataIndex]['network'] = value[1]
                    dataIndex+=1

    print(metricsData) 
    return metricsData

                

In [54]:
getCurrentJsonData()

{'results': [{'statement_id': 0, 'series': [{'name': 'cpu/usage_rate', 'tags': {'container_name': 'attachment', 'namespace_name': 'us', 'pod_name': 'attachment-1358228146-ftllw'}, 'columns': ['time', 'sum'], 'values': [['2017-06-08T03:20:00Z', 2]]}]}]}
{'results': [{'statement_id': 0, 'series': [{'name': 'memory/usage', 'tags': {'container_name': 'attachment', 'namespace_name': 'us', 'pod_name': 'attachment-1358228146-ftllw'}, 'columns': ['time', 'max'], 'values': [['2017-06-08T03:20:00Z', 5568794624]]}]}]}
{'results': [{'statement_id': 0, 'series': [{'name': 'filesystem/usage', 'tags': {'container_name': 'attachment', 'namespace_name': 'us', 'pod_name': 'attachment-1358228146-ftllw'}, 'columns': ['time', 'sum'], 'values': [['2017-06-08T03:20:00Z', 842362880]]}]}]}
{'results': [{'statement_id': 0, 'series': [{'name': 'network/tx_rate', 'tags': {'container_name': '', 'namespace_name': 'us', 'pod_name': 'attachment-1358228146-ftllw'}, 'columns': ['time', 'sum'], 'values': [['2017-06-08T0

{'pods': [{'data': [{'cpu': 2,
     'fileSystem': 842362880,
     'memory': 5568794624,
     'network': 4315.6123046875,
     'time': '2017-06-08T03:20:00Z'}],
   'podName': 'attachment-1358228146-ftllw'}]}

In [55]:
getJsonData(60)

{'results': [{'statement_id': 0, 'series': [{'name': 'cpu/usage_rate', 'tags': {'container_name': 'attachment', 'namespace_name': 'us', 'pod_name': 'attachment-1358228146-ftllw'}, 'columns': ['time', 'sum'], 'values': [['2017-06-08T03:21:00Z', 7]]}]}]}
{'results': [{'statement_id': 0, 'series': [{'name': 'memory/usage', 'tags': {'container_name': 'attachment', 'namespace_name': 'us', 'pod_name': 'attachment-1358228146-ftllw'}, 'columns': ['time', 'max'], 'values': [['2017-06-08T03:21:00Z', 5562843136]]}]}]}
{'results': [{'statement_id': 0, 'series': [{'name': 'filesystem/usage', 'tags': {'container_name': 'attachment', 'namespace_name': 'us', 'pod_name': 'attachment-1358228146-ftllw'}, 'columns': ['time', 'sum'], 'values': [['2017-06-08T03:21:00Z', 1685016576]]}]}]}
{'results': [{'statement_id': 0, 'series': [{'name': 'network/tx_rate', 'tags': {'container_name': '', 'namespace_name': 'us', 'pod_name': 'attachment-1358228146-ftllw'}, 'columns': ['time', 'sum'], 'values': [['2017-06-08T

{'pods': [{'data': [{'cpu': 7,
     'fileSystem': 1685016576,
     'memory': 5562843136,
     'network': 25785.89453125,
     'time': '2017-06-08T03:21:00Z'}],
   'podName': 'attachment-1358228146-ftllw'}]}

In [49]:
getCurrentJsonData()

{'pods': []}


{'pods': []}